<a href="https://colab.research.google.com/github/lingchm/datascience/blob/master/exercises/low_rank_matrix_completion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low Rank Matrix Completion

*convex optimization*, *regularization*, *proximal-gradient algorithm*, *SVD*

**Problem**

Given a matrix $Y \in \mathbb{R}^{MXN}$ with missing entries, complete the matrix such that it is the lowest possible rank.

NOTE: This can be used to solve the Netflix $1M prize problem!

**Method**

We formulate the following "matrix LASSO" problem. The lowest rank requirement is achieved by regularizing the matrix's nuclear norm.

\begin{equation}
\min_{X \in \mathbb{R}^{MxN}} \frac{\delta}{2} \sum_{(m,n)\in I} (Y_{m,n} - X_{m,n})^2 + ||X||_{nn}
\end{equation}

The proximal operator of the spectral norm has closed form solution. We know that the nuclear norm is the dual of the spectral norm. The two can be connected by Moreau decomposiion,
\begin{equation}
x = prox_{\alpha, f}(x) + \alpha prox_{\alpha^{-1}, f^*}(x / \alpha)
\end{equation}
where $f^*$ is the fenchel conjugate of $f$. 

Hence, a handy way is using proximal-gradient algorithm

**References**
* https://gclinderman.github.io/blog/statistics/probability/matrix/completion/2018/07/08/matrix-completion.html
* Credits to Dr. Justin Romberg for designing this problem.
* Steve Brunton: https://www.youtube.com/watch?v=sooj-_bXWgk&list=PLMrJAkhIeNNSVjnsviglFoY2nXildDCcv&index=9

In [ ]:
import numpy as np

In [1]:
# set up array 
Y = np.asarray([[10,  0,  0,  6,  4,  4, 12,  0],
       [12,  0,  6,  0,  6,  0, 18, 12],
       [ 0, 10,  4, 10,  0,  4,  0, 10],
       [ 0,  0,  2,  4,  0,  2,  6,  4],
       [ 0,  4,  2,  0,  0,  2,  6,  4],
       [10,  0,  4,  6,  0,  4, 12,  0],
       [10,  0,  4,  0,  4,  0, 12,  6],
       [ 6, 10,  0, 10,  4,  0, 12,  0]])
masked = np.asarray([[1, 0, 0, 1, 1, 1, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 1],
       [0, 1, 1, 1, 0, 1, 0, 1],
       [0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 0, 0, 1, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 1],
       [1, 1, 0, 1, 1, 0, 1, 0]])

In [2]:
# implement proximal gradient algorithm
def prox_sn(Z):
    u, s, vh = np.linalg.svd(Z, full_matrices=False)
    for i in range(s.shape[0]): 
        if s[i] >= 1:
            s[i] = 1    
    s = np.diag(s)
    return u @ s @ vh
    
def prox_nn(Z, alpha):
    return Z - alpha * prox_sn(Z / alpha)
    
def gradf(delta, X, Y):
    return delta * (X - Y)

def proximal_gradient(Y, masked, alpha=0.01, delta=100, tol=1e-3, max_iter=10000):
    X0 = np.zeros(Y.shape)
    k, Xk, Xk_ = 0, X0, X0 + 1
    while np.linalg.norm(Xk - Xk_) > tol and k < max_iter:
        #f = delta / 2 * np.linalg.norm(Y - Xk)**2 + np.linalg.norm(Xk, ord='nuc')
        Xk_ = Xk * 1
        Xk = prox_nn(Xk - alpha * gradf(delta, Xk, Y) * masked, alpha)
        k += 1
        #print("Functional value:", f)
    print("Number of iterations:", k)
    return Xk

In [3]:
# test
Y_ = proximal_gradient(Y, masked)
print("Estimated Y:")
print(np.round(Y_))
print("Rank", np.linalg.matrix_rank(Y_))

Number of iterations: 7376
Estimated Y:
[[10.  5.  4.  6.  4.  4. 12.  6.]
 [12. 11.  6. 12.  6.  6. 18. 12.]
 [ 6. 10.  4. 10.  4.  4. 12. 10.]
 [ 4.  4.  2.  4.  2.  2.  6.  4.]
 [ 4.  4.  2.  4.  2.  2.  6.  4.]
 [10.  5.  4.  6.  4.  4. 12.  6.]
 [10.  5.  4.  6.  4.  4. 12.  6.]
 [ 6. 10.  4. 10.  4.  4. 12. 10.]]
Rank 3
